# Lending Club Data Exploration

### choosing the relevant  data from the data base
In this notebook we are going to take a first glance of the data and choosing the rellavant parameters and loans to answer our quotation.



In [103]:
##Importing required packages 
if(!require(DBI)){install.packages("DBI"); require(DBI)}
if(!require(RSQLite)){install.packages("RSQLite"); require(RSQLite)}
require(dplyr)

In [104]:
##WorkPath
#DATA_PATH = "D:/Google Drive/Final Project/Data"
DB_FILE = ("loans.db") 

In [105]:
##############################################################################
##########                DATABASE FUNCTIONS                     #############
##############################################################################
            
#### Read function to import data from the SQL to a pandas dataframe.
readSQL <- function(query, db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    df <- dbGetQuery(con, query)
    return(df)
}
#### Write a pandas dataframe into an SQL table. Use overwrite=True if you want to delete 
#### first a pre-existent table with the same name. Use append=True if you want to append
#### the data in the dataframe to a pre-existent table.
writeSQL <- function(df,tablename,overwrite=FALSE, append=FALSE,db=DB_FILE) {
    require(DBI)
    require(RSQLite)
    con <- dbConnect(SQLite(), DB_FILE)
    dbWriteTable(con,tablename,df,overwrite,append)
}
####

In [204]:
loans = readSQL("SELECT * FROM v_loans")

In [205]:
dim(loans)

[1] 887382     74

we can see that we have 887383 loans and 74 fitchers.
first of all, lets take of all the unusefall fitchers.

#### data description


hare is a brief explanation on the different  data's parameters

In [108]:
#make a coppy
loansdata <- loans

In [109]:

if(!require(readxl)){install.packages("readxl"); require(readxl)}
# Load the Excel workbook
##path <- "D:/Google Drive/Final Project/"
##WorkPath
#path = "C:/Users/Kobi/Google Drive/Final Project/"
excel_file <- paste0("/lcdatadictionary.xlsx")
# see available tabs
excel_sheets(paste0(path, excel_file))
meta_loan_stats <- read_excel(paste0(path, excel_file), sheet = "LoanStats")

ERROR: Error: 'C:/Users/Kobi/Google Drive/Final Project//lcdatadictionary.xlsx' does not exist.


In [110]:
meta_loan_stats[,1:2]

ERROR: Error in eval(expr, envir, enclos): object 'meta_loan_stats' not found


In [ ]:
summary(loans)

Sorting the parameters by their type

In [ ]:
split(names(loans),sapply(loans, function(x) paste(class(x), collapse=" ")))

In [ ]:
## numeric variables
num_vars <- 
  loans %>% 
  sapply(is.numeric) %>% 
  which() %>% 
  names()
num_vars
length(num_vars)

#### Fixing the dates

In [111]:
chr_to_date_vars <- 
  c("issue_d", "last_pymnt_d", "last_credit_pull_d",
    "next_pymnt_d", "earliest_cr_line")

loans %>%
  select_(.dots = chr_to_date_vars) %>%
  str()

'data.frame':	887382 obs. of  5 variables:
 $ issue_d           : chr  "Dec-2011" "Dec-2011" "Dec-2011" "Dec-2011" ...
 $ last_pymnt_d      : chr  "Jan-2015" "Apr-2013" "Jun-2014" "Jan-2015" ...
 $ last_credit_pull_d: chr  "Jan-2016" "Sep-2013" "Jan-2016" "Jan-2015" ...
 $ next_pymnt_d      : chr  NA NA NA NA ...
 $ earliest_cr_line  : chr  "Jan-1985" "Apr-1999" "Nov-2001" "Feb-1996" ...


In [144]:

loans <- loansdata

In [113]:
##convert-date as string to Date Variable
convert_date <- function(x){
  as.Date(paste0("01-", x), format = "%d-%b-%Y")
  } 
lct <- Sys.getlocale("LC_TIME"); Sys.setlocale("LC_TIME", "C")
loans <-
  loans %>%
  mutate_at(.funs = funs(convert_date), .vars = chr_to_date_vars)
Sys.setlocale("LC_TIME", lct)

[1] "C"

[1] "Hebrew_Israel.1255"

In [114]:
## categorical variables
cv <- setdiff(names(loans),c(chr_to_date_vars,num_vars))
cv

[1] "verification_status"       "loan_status"              
 [3] "url"                       "loanDesc"                 
 [5] "purpose"                   "title"                    
 [7] "verification_status_joint" "grade"                    
 [9] "sub_grade"                 "emp_title"                
[11] "emp_length"                "zip_code"                 
[13] "addr_state"                "home_ownership"

In [115]:
loans %>%
  select_(.dots = cv) %>%
  str()

'data.frame':	887382 obs. of  14 variables:
 $ verification_status      : chr  "Verified" "Source Verified" "Not Verified" "Source Verified" ...
 $ loan_status              : chr  "Fully Paid" "Charged Off" "Fully Paid" "Fully Paid" ...
 $ url                      : chr  "https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501" "https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430" "https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175" "https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863" ...
 $ loanDesc                 : chr  "  Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>" "  Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to ha"| __truncated__ NA "  Borrower added on 12/21/11 > to pay for property tax (borrow from friend, need to pay back) & central A/C nee"| __truncated__ ...
 $ purpose                  : chr  "credit_card" "car" "sma

#### some more data checking

In [116]:
## tableone package itself
if(!require(tableone)){install.packages("tableone"); require(tableone)}

In [117]:
## Create a TableOne object
tab <- CreateTableOne(vars = names(loans), data = loans)

Warning message in CreateTableOne(vars = names(loans), data = loans):
"Dropping variable(s) issue_d last_pymnt_d next_pymnt_d last_credit_pull_d earliest_cr_line  due to unsupported class.
"

In [118]:
summary(tab)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [119]:
## Create a TableOne object
tab2 <- CreateTableOne(vars = num_vars, data = loans)

In [120]:
summary(tab2)


     ### Summary of continuous variables ###

strata: Overall
                                n  miss p.miss  mean    sd median   p25   p75
loan_id                     9e+05 0e+00  0e+00 3e+07 2e+07  3e+07 9e+06 5e+07
member_id                   9e+05 0e+00  0e+00 4e+07 2e+07  4e+07 1e+07 6e+07
pymnt_plan                  9e+05 0e+00  0e+00 1e-05 3e-03  0e+00 0e+00 0e+00
policy_code                 9e+05 0e+00  0e+00 1e+00 0e+00  1e+00 1e+00 1e+00
application_type            9e+05 0e+00  0e+00 6e-04 2e-02  0e+00 0e+00 0e+00
initial_list_status         9e+05 0e+00  0e+00 5e-01 5e-01  1e+00 0e+00 1e+00
loan_amnt                   9e+05 0e+00  0e+00 1e+04 8e+03  1e+04 8e+03 2e+04
funded_amnt                 9e+05 0e+00  0e+00 1e+04 8e+03  1e+04 8e+03 2e+04
funded_amnt_inv             9e+05 0e+00  0e+00 1e+04 8e+03  1e+04 8e+03 2e+04
term                        9e+05 0e+00  0e+00 3e-01 5e-01  0e+00 0e+00 1e+00
int_rate                    9e+05 0e+00  0e+00 1e+01 4e+00  1e+01 1e+01 2e+01
i

In [121]:
## funModeling package
if(!require(funModeling)){install.packages("funModeling"); require(funModeling)}

Loading required package: funModeling
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
"there is no package called 'funModeling'"Warning message:
"unable to access index for repository https://mran.microsoft.com/snapshot/2017-09-01/src/contrib:
  cannot open destfile 'C:\Users\User\AppData\Local\Temp\RtmpmqRett\file208836531546', reason 'No such file or directory'"Warning message:
"unable to access index for repository http://www.stats.ox.ac.uk/pub/RWin/src/contrib:
  cannot open destfile 'C:\Users\User\AppData\Local\Temp\RtmpmqRett\file2088107526bf', reason 'No such file or directory'"Warning message:
"package 'funModeling' is not available (for R version 3.4.3)"Warning message in gzfile(file, mode):
"cannot open compressed file 'C:\Users\User\AppData\Local\Temp\RtmpmqRett/libloc_191_2e9ef8b0.rds', probable reason 'No such file or directory'"

ERROR: Error in gzfile(file, mode): cannot open the connection


In [122]:
meta_loans <- funModeling::df_status(loans, print_results = FALSE)

ERROR: Error in loadNamespace(name): there is no package called 'funModeling'


In [ ]:
meta_loans%>%
  knitr::kable()

In [ ]:
meta_loans <-
  meta_loans %>%
  mutate(uniq_rat = unique / nrow(loans))

meta_loans %>%
  select(variable, unique, uniq_rat) %>%
  mutate(unique = unique, uniq_rat = scales::percent(uniq_rat))

In [ ]:
meta_loans %>%
  select(variable, p_zeros, p_na, unique) %>%
  filter_(~ variable %in% num_vars)

In [ ]:
meta_loans %>%
  select(variable, p_zeros, p_na, unique) %>% arrange(desc(unique))

<h3>Excluding Columns (1)</h3>
<ol>
<li>It seems the data has<strong> two unique identifiers</strong> <span style="text-decoration: underline; color: #0000ff;">loan_id</span> and <span style="text-decoration: underline; color: #0000ff;">member_id</span>.<br /> We using them only for joins between loans and loaners tables. So we can drop one of them.</li>
<li>The <span style="text-decoration: underline; color: #0000ff;">Url</span> column is 100% unique. <br /> By Lending Club Definition File - URL for the LC page with listing data.<br /> This data is not meaningfull for model purposes.</li>
<li><span style="text-decoration: underline; color: #0000ff;">Policy code</span> has only one unique value.<br /> By Lending Club Definition File <br />publicly available policy_code=1<br /> new products not publicly available policy_code=2<br /> So we don't have rows with policy_code=2.</li>
<li>An attribute <span style="text-decoration: underline; color: #0000ff;">emp_title </span>has over 30% unique values, it seems borrowers are free to describe their employment title. <br /><em><span style="text-decoration: underline;">Note:</span> </em>In a more sophisticated modeling , may be used to checking the strings for &ldquo;indicator words&rdquo; that may be associated with an honest or a dishonest (credible / non-credible) candidate.</li>
</ol>

In [206]:
excludeColumns <- function(data,colv) {
    for(n in colv) {
        data[[n]] <- NULL
    }
    return(data)
}
exclude <- c("member_id","url","emp_title","policy_code")
ncol(loans)
loans <- excludeColumns(loans,exclude)
ncol(loans)

[1] 74

[1] 70

In [207]:
excludeUniqueValue <- function(data) {
    nm <- names(data)
    res <- NULL
    for(n in nm) {
        if(length(unique(data[[n]]))==1) {
            data[[n]] <- NULL
            res <- c(res, n)
            print(n)
        }
    }
    if(length(res) > 0) {
        message("The following variables had only one unique values and were removed")
        message(res)
    }
    return(data)
}

In [208]:
meta_loans %>%
  select(variable, p_zeros, p_na, unique) %>% arrange(desc(p_na))

ERROR: Error in eval(lhs, parent, parent): object 'meta_loans' not found


<p>We see that the columns <strong><em>verification_status_joint,annual_inc_joint,dti_joint</em></strong> has a <span style="color: #ff0000;"><strong>99% NA ratio</strong></span><br />This Columns By their LC Definition indicate those are columns which describe a <span style="text-decoration: underline;">joint application</span> loans.<br />We will check for the ratio of joint apllications in our dataset:</p>

In [209]:
loans %>% group_by(application_type) %>% summarise(N = n())%>%select(application_type,N)%>%mutate(app_rat = N / nrow(loans))

application_type,N,app_rat
0,886871,0.9994241488
1,511,0.0005758512


<p>As we can see , the 99.4% of our data is for single application, therefore we choosing to eliminate rows for the joint application loans to avoid outliers of this population.</p>
<p><span style="color: #000000;"><span style="text-decoration: underline; color: #ff0000;"><strong>Assumption</strong></span> -</span> Joint applications are statistically different than single applications.</p>
<p><strong><span style="text-decoration: underline;"><span style="color: #ff0000; text-decoration: underline;">Actions:</span></span></strong></p>
<ol>
<li>Remove these 3 columns:&nbsp;<strong><em>verification_status_joint,annual_inc_joint,dti_joint</em></strong></li>
<li>Remove rows for Join applications (application_type=1)</li>
<li>Because we removed all the rows with&nbsp;application_type=1 there is only one unique value so we will remove this column.</li>
</ol>

In [210]:
exclude <- c("verification_status_joint","annual_inc_joint","dti_joint")
ncol(loans)
loans <- excludeColumns(loans,exclude)
ncol(loans)
nrow(loans)
loans <- loans %>% filter(application_type == 0)
nrow(loans)
loans<-excludeUniqueValue(loans)

[1] 70

[1] 67

[1] 887382

[1] 886871

[1] "application_type"


The following variables had only one unique values and were removed
application_type


<h3>Columns which are indications of the outcome - determind by Lending Club after the company evaluations of the credit risk. In our work, we want to estimate the credit risk by ourselves based on the data we have. </h3>
<ol>
<li>grade</li>
<li>sub_grade</li>
<li>int_rate</li>
</ol>


In [211]:
exclude <- c("grade","sub_grade","int_rate")
ncol(loans)
loans <- excludeColumns(loans,exclude)
ncol(loans)

[1] 66

[1] 63

In [212]:
#Recreate Meta Loans
meta_loans <- funModeling::df_status(loans, print_results = FALSE)
meta_loans <-
  meta_loans %>%
  mutate(uniq_rat = unique / nrow(loans))

ERROR: Error in loadNamespace(name): there is no package called 'funModeling'


In [213]:
meta_loans %>%
  select(variable, p_zeros, p_na, unique) %>% arrange(desc(p_na))

ERROR: Error in eval(lhs, parent, parent): object 'meta_loans' not found


<h1>Defining default</h1>
<p>Our ultimate goal is the prediction of loan defaults.</p>
<p>The variable loan status seems to be an indicator of the current state a particular loan is in.</p>
<p>The different loan statuses in our data</p>

In [214]:
loans %>%
  group_by(loan_status) %>%
  summarise(count = n(), rel_count = count/nrow(loans))

loan_status,count,rel_count
Charged Off,45248,0.0510198214
Current,601340,0.6780467509
Default,1219,0.0013744953
Does not meet the credit policy. Status:Charged Off,761,0.0008580729
Does not meet the credit policy. Status:Fully Paid,1988,0.0022415887
Fully Paid,207723,0.2342200839
In Grace Period,6250,0.0070472481
Issued,8396,0.0094669913
Late (16-30 days),2357,0.0026576582
Late (31-120 days),11589,0.0130672894


<p>It is not immediately obvious what the different values stand for, so we refer to Lending Club&rsquo;s documentation about &ldquo;<a href="https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-">What do the different Note statuses mean?</a>&rdquo;</p>
<ul>
<li>Fully Paid: Loan has been fully repaid, either at the expiration of the 3- or 5-year year term or as a result of a prepayment.</li>
<li>Current: Loan is up to date on all outstanding payments.</li>
<li>Does not meet the credit policy. Status:Fully Paid: No explanation but see &ldquo;fully paid&rdquo;.</li>
<li>Issued: New loan that has passed all Lending Club reviews, received full funding, and has been issued.</li>
<li>Charged Off: Loan for which there is no longer a reasonable expectation of further payments. Generally, Charge Off occurs no later than 30 days after the Default status is reached. Upon Charge Off, the remaining principal balance of the Note is deducted from the account balance. Learn more about the&nbsp;<a href="https://help.lendingclub.com/hc/en-us/articles/216127747">difference between &ldquo;default&rdquo; and &ldquo;charge off&rdquo;</a>.</li>
<li>Does not meet the credit policy. Status:Charged Off: No explanation but see &ldquo;Charged Off&rdquo;</li>
<li>Late (31-120 days): Loan has not been current for 31 to 120 days.</li>
<li>In Grace Period: Loan is past due but within the 15-day grace period.</li>
<li>Late (16-30 days): Loan has not been current for 16 to 30 days.</li>
<li>Default: Loan has not been current for 121 days or more.</li>
</ul>
<p>Given above information, we will define a default as follows.</p>
<b>We can see that 207,724 from the loans are Fully Paid and 45,248 are Charged Off.
Lending Club defined Charged Off loans as loans that did not paid and Fully paid loans as loans that fully paid.
Other loans, are loans that their status is not steady yet.

In [215]:
copy1<-loans

In [216]:
nrow(loans)
status<-c("Current", "Default","In Grace Period", "Issued", "Late (16-30 days)",
          "Late (16-30 days)", "Late (31-120 days)"  ) 


loans <-
loans %>% filter(loan_status %in% status)
nrow(loans)

[1] 886871

[1] 631151

In [217]:
Charged_Off <- 
  c( "Charged Off",
    "Does not meet the credit policy. Status:Charged Off")


In [218]:
loans <-
  loans %>%
  mutate(default = ifelse(!(loan_status %in% Charged_Off), FALSE, TRUE))


<p>Remove loan_status column because it has been used to define target variable</p>

In [219]:
loans <- loans %>% select(-one_of("loan_status"))

In [220]:
split(names(loans),sapply(loans, function(x) paste(class(x), collapse=" ")))

$character
 [1] "verification_status" "issue_d"             "loanDesc"           
 [4] "purpose"             "title"               "emp_length"         
 [7] "zip_code"            "addr_state"          "last_pymnt_d"       
[10] "next_pymnt_d"        "last_credit_pull_d"  "home_ownership"     
[13] "earliest_cr_line"   

$integer
 [1] "loan_id"                     "pymnt_plan"                 
 [3] "initial_list_status"         "loan_amnt"                  
 [5] "funded_amnt"                 "term"                       
 [7] "collections_12_mths_ex_med"  "mths_since_last_major_derog"
 [9] "delinq_2yrs"                 "acc_now_delinq"             
[11] "inq_last_6mths"              "mths_since_last_delinq"     
[13] "mths_since_last_record"      "open_acc"                   
[15] "pub_rec"                     "total_acc"                  
[17] "tot_coll_amt"                "open_acc_6m"                
[19] "open_il_6m"                  "open_il_12m"                
[21] "open_il_24m"                 "mths_since_rcnt_il"         
[23] "total_bal_il"                "open_rv_12m"                
[25] "open_rv_24m"                 "max_bal_bc"                 
[27] "inq_fi"                      "total_cu_tl"                
[29] "inq_last_12m"               

$logical
[1] "default"

$numeric
 [1] "funded_amnt_inv"         "installment"            
 [3] "out_prncp"               "out_prncp_inv"          
 [5] "total_pymnt"             "total_pymnt_inv"        
 [7] "total_rec_prncp"         "total_rec_int"          
 [9] "total_rec_late_fee"      "recoveries"             
[11] "collection_recovery_fee" "last_pymnt_amnt"        
[13] "annual_inc"              "dti"                    
[15] "revol_bal"               "revol_util"             
[17] "tot_cur_bal"             "il_util"                
[19] "all_util"                "total_rev_hi_lim"

In [137]:
#Recreate Meta Loans
meta_loans <- funModeling::df_status(loans, print_results = FALSE)
meta_loans <-
  meta_loans %>%
  mutate(uniq_rat = unique / nrow(loans)) %>%
  mutate(unique = unique, uniq_rat = scales::percent(uniq_rat))

ERROR: Error in loadNamespace(name): there is no package called 'funModeling'


In [138]:
cv <- c('verification_status','loanDesc', 'purpose' ,'title','emp_length','zip_code', 'addr_state', 'home_ownership')
meta_loans %>%
  select(variable, p_zeros, p_na, unique,uniq_rat) %>% arrange(desc(p_na))  %>% filter_(~ variable %in% cv)

ERROR: Error in eval(lhs, parent, parent): object 'meta_loans' not found


<h3>Excluding Columns (2)</h3>
<ol>
<li>An attribute <span style="text-decoration: underline; color: #0000ff;">loanDesc </span>has over 30% unique values, it seems borrowers are free to describe their loan application. <br /><em><span style="text-decoration: underline;">Note:</span> </em>In a more sophisticated modeling , may be used to checking the strings for &ldquo;indicator words&rdquo; that may be associated with an honest or a dishonest candidate as we have the purpose of the loan.</li>
<li>An attribute <span style="text-decoration: underline; color: #0000ff;">title </span>has a lot of unique values, it seems borrowers are free to describe their title for the loan application. <br /><em><span style="text-decoration: underline;">Note:</span> </em>In a more sophisticated modeling , may be used to checking the strings for &ldquo;indicator words&rdquo; that may be associated with an honest or a dishonest candidate as we have the purpose of the loan.</li>
</ol>

In [221]:
exclude <- c("loanDesc","title")
ncol(loans)
loans <- excludeColumns(loans,exclude)
ncol(loans)

[1] 63

[1] 61

In [222]:
cv <- c('verification_status', 'purpose' ,'emp_length', 'home_ownership')
tab <- CreateTableOne(vars = cv, data = loans)

In [223]:
summary(tab)


     ### Summary of categorical variables ### 

strata: Overall
                 var      n miss p.miss              level   freq percent
 verification_status 631151    0    0.0       Not Verified 178739    28.3
                                           Source Verified 255125    40.4
                                                  Verified 197287    31.3
                                                                         
             purpose 631151    0    0.0                car   5152     0.8
                                               credit_card 155653    24.7
                                        debt_consolidation 374418    59.3
                                               educational      1     0.0
                                          home_improvement  36661     5.8
                                                     house   2011     0.3
                                            major_purchase  10887     1.7
                                               

In [ ]:
##LoansBeforeEncoding
loansBE <- loans
writeSQL(loansBE, "loans_beforeDataSet")
##loansBE = readSQL("SELECT * FROM loans_beforeDataSet")

In [ ]:
## one-hot-encoding...
one_hot_encoding <- function(data,id,category,prefix) {
    require(dplyr)
    cat <- levels(factor(data[[category]]))
    res <- data %>% group_by_(id) %>% select_(id) %>% distinct_()
    for (l in 1:length(cat)) {
        #cname <- paste(prefix,l,sep="_")
        #data[[cname]] <- ifelse(data[[category]]==cat[l],1,0)
        #cc1 <- data %>% group_by_(id) %>% summarise_(cname=max(cname)) %>% select_(id,cname)
        #res <- cbind(res, cc1[,2])
        cname <- paste(prefix,l,sep="_")
        ncat <- "newcat"
        data[[ncat]] <- ifelse(data[[category]]==cat[l],1,0)
        cc1 <- data %>% select_(id,ncat) %>% group_by_(id) %>% 
               summarise(newcat=max(newcat))
        res[[cname]] <- cc1$newcat
    }
    return(res)
}

In [ ]:
loans <- loansBE
nrow(loans)
head(loans)

In [ ]:
head(one_hot_encoding(data=loans,id="loan_id",category="verification_status",prefix="verif"))


In [ ]:
loans <- inner_join(loans,one_hot_encoding(data=loans,id="loan_id",category="verification_status",prefix="verif"))

In [ ]:
nrow(loans)
head(loans)

In [ ]:
head(one_hot_encoding(data=loans,id="loan_id",category="purpose",prefix="purpose"))


In [ ]:
loans <- inner_join(loans,one_hot_encoding(data=loans,id="loan_id",category="purpose",prefix="purpose"))

In [ ]:
nrow(loans)
head(loans)

In [ ]:
head(one_hot_encoding(data=loans,id="loan_id",category="emp_length",prefix="seniority"))


In [ ]:
loans <- inner_join(loans,one_hot_encoding(data=loans,id="loan_id",category="emp_length",prefix="seniority"))

In [ ]:
nrow(loans)
head(loans)

In [ ]:
head(one_hot_encoding(data=loans,id="loan_id",category="home_ownership",prefix="owner_status"))

In [ ]:
loans <- inner_join(loans,one_hot_encoding(data=loans,id="loan_id",category="home_ownership",prefix="owner_status"))

In [ ]:
nrow(loans)
head(loans)

In [ ]:
ncol(loans)

In [ ]:
funModeling::df_status(loans, print_results = TRUE)

In [ ]:
loans <- excludeUniqueValue(loans)

In [ ]:
num_vars <- 
  loans %>% 
  sapply(is.numeric) %>% 
  which() %>% 
  names()

In [ ]:
num_vars

In [ ]:
tab <- CreateTableOne(vars = num_vars, data = loans)
summary(tab)

## future work

<h3>Columns to remove - Maybe on LATER Stage - after missingnes checks</h3>
<ul>
<li>desc: high NA ratio</li>
<li>title: high amount of unique values</li>
<li>next_pymnt_d: high NA ratio</li>
<li>open_acc_6m: high NA ratio</li>
<li>open_il_6m: high NA ratio</li>
<li>open_il_12m: high NA ratio</li>
<li>open_il_24m: high NA ratio</li>
<li>mths_since_rcnt_il: high NA ratio</li>
<li>total_bal_il: high NA ratio</li>
<li>il_util: high NA ratio</li>
<li>open_rv_12m: high NA ratio</li>
<li>open_rv_24m: high NA ratio</li>
<li>max_bal_bc: high NA ratio</li>
<li>all_util: high NA ratio</li>
<li>total_rev_hi_lim: high NA ratio</li>
<li>inq_fi: high NA ratio</li>
<li>total_cu_tl: high NA ratio</li>
<li>inq_last_12m: high NA ratio</li>
</ul>

In [ ]:

exclude <- c("loanDesc", "title", "open_acc_6m", "open_il_6m", 
   "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il", 
   "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util",
    "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m",
    "verification_status_joint", "next_pymnt_d")


In [ ]:
##saving the file ,for future work!

write.csv(loans, file = "loans_db.csv",row.names=FALSE)
writeSQL(loans, "loans_dataset")